In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import numpy as np
import random
import os

from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
trainPath = r'dataset\dataSplit\train'
validationPath = r'dataset\dataSplit\val'
testPath = r'dataset\dataSplit\test'

In [ ]:
def count_files_in_directory(directory_path):
    file_count = 0

    for root, dirs, files in os.walk(directory_path):
        file_count += len(files)

    return file_count

train_files_count = count_files_in_directory(trainPath)
validation_files_count = count_files_in_directory(validationPath)
test_files_count = count_files_in_directory(testPath)

print(f"Total Data training: {train_files_count}")
print(f"Total Data validation: {validation_files_count}")
print(f"Total Data test: {test_files_count}")

In [ ]:
def count_files_per_class(directory_path):
    class_file_count = {}
    classes = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]

    for class_name in classes:
        class_path = os.path.join(directory_path, class_name)
        file_count = len([f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))])
        class_file_count[class_name] = file_count

    return class_file_count

train_files_per_class = count_files_per_class(trainPath)
validation_files_per_class = count_files_per_class(validationPath)
test_files_per_class = count_files_per_class(testPath)

print("Data Training per class:")
for class_name, count in train_files_per_class.items():
    print(f"{class_name}: {count}")

print("\nData Validation per class:")
for class_name, count in validation_files_per_class.items():
    print(f"{class_name}: {count}")

print("\nData Test per class:")
for class_name, count in test_files_per_class.items():
    print(f"{class_name}: {count}")

In [ ]:
def show_sample_images(directory_path):
    classes = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]
    plt.figure(figsize=(15, 10))

    for i, class_name in enumerate(classes):
        class_path = os.path.join(directory_path, class_name)
        files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
        sample_file = random.choice(files)
        sample_path = os.path.join(class_path, sample_file)

        img = mpimg.imread(sample_path)
        plt.subplot(3, 3, i + 1)
        plt.imshow(img)
        plt.title(class_name)
        plt.axis('off')

    plt.show()

show_sample_images(trainPath)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

img_size = (224, 224)
batch_size = 32

train_generator = train_datagen.flow_from_directory(
                                            trainPath,
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    shuffle=False,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                            )

validation_generator=val_datagen.flow_from_directory(
                                            validationPath,
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    shuffle=False,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                )

test_generator=test_datagen.flow_from_directory(
                                            testPath,
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    shuffle=False,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                )

In [ ]:
def show_augmented_samples(generator):
    images, labels = next(generator)
    class_indices = {v: k for k, v in generator.class_indices.items()}
    plt.figure(figsize=(15, 15))

    for i in range(9):
        plt.subplot(3, 3, i + 1)
        img = images[i]
        label = np.argmax(labels[i])
        class_name = class_indices[label]
        plt.imshow(img)
        plt.title(f'Class: {class_name}')
        plt.axis('off')

    plt.show()

show_augmented_samples(train_generator)

In [ ]:
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss',
                                    factor=0.5, patience=3,
                                    verbose=1, mode='min',
                                    min_delta=0.0001, min_lr=1e-3,
                                    restore_best_weights=True)

In [ ]:
pre_trained_model_ResNet50V2 = ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=(244, 244, 3),
)

pre_trained_model_ResNet50V2.trainable = False

In [ ]:
model_ResNet50V2 = tf.keras.Sequential([
            pre_trained_model_ResNet50V2,
            layers.GlobalAveragePooling2D(),
            layers.Dense(1024, activation='relu'),
            layers.Dense(512, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(9, activation='softmax')
])

In [ ]:
model_ResNet50V2.compile(optimizer = Adam(learning_rate=1e-3),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model_ResNet50V2.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 50,
            verbose=1,
            callbacks=[reduceLROnPlat])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
model_ResNet50V2.save('saved-models/model_ResNet50V2.h5')

In [ ]:
model_B6_load = load_model('saved-models/model_ResNet50V2.h5')

In [ ]:
Y_pred = model_B6_load.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

In [ ]:
conf_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average=None)
recall = recall_score(y_true, y_pred, average=None)

class_report = classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys())

print("Confusion Matrix:\n", conf_matrix)
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: ", precision)
print("Recall: ", recall)
print("\nClassification Report:\n", class_report)